# Dependence Networks

From [http://openonlinecourses.com/causalanalysis/ReviewIndependence.asp](http://openonlinecourses.com/causalanalysis/ReviewIndependence.asp).

## Question 1.1 complete independence

We are testing if `ALL` 3 variables (MD, RN, Complaint) are completely independent.

Load data.

In [1]:
import pandas as pd

df = pd.DataFrame({
    'MD': ['George'] * 4 + ['Smith'] * 4,
    'RN': ['Jim', 'Jim', 'Jill', 'Jill'] * 2,
    'Complaint': ['Yes', 'No'] * 4,
    'Observed': [53, 424, 11, 37, 0, 18, 4, 139]
})

df

,MD,RN,Complaint,Observed
0,George,Jim,Yes,53
1,George,Jim,No,424
2,George,Jill,Yes,11
3,George,Jill,No,37
4,Smith,Jim,Yes,0
5,Smith,Jim,No,18
6,Smith,Jill,Yes,4
7,Smith,Jill,No,139


Compute the `expected` (here, called `Predicted`) values.

In [2]:
from functools import reduce

def get_count(field, val):
    return df[df[field] == val]['Observed'].sum()

def get_expected(r):
    counts = [get_count(f, r[f]) for f in ['MD', 'RN', 'Complaint']]
    n = df['Observed'].sum()
    expected = reduce(lambda a, b: a * b, counts) / (n * n)
    return expected
    
df['Predicted'] = df.apply(get_expected, axis=1)
df

,MD,RN,Complaint,Observed,Predicted
0,George,Jim,Yes,53,37.551318
1,George,Jim,No,424,341.275213
2,George,Jill,Yes,11,14.489498
3,George,Jill,No,37,131.683971
4,Smith,Jim,Yes,0,11.515737
5,Smith,Jim,No,18,104.657732
6,Smith,Jill,Yes,4,4.443446
7,Smith,Jill,No,139,40.383084


Compute the $\chi^2$ value.

In [3]:
import numpy as np

def diff_sq(r):
    return np.power(r['Observed'] - r['Predicted'], 2) / r['Predicted']

chi_sq = df.apply(diff_sq, axis=1).sum()
chi_sq

419.4679873479184

Compute the degrees of freedom.

In [4]:
def get_dof():
    num_values = [len(df[f].unique()) for f in ['MD', 'RN', 'Complaint']]
    IJK = reduce(lambda a, b: a * b, num_values)
    dof = [IJK] + [-n for n in num_values] + [2]
    
    return sum(dof)

dof = get_dof()
dof

4

There's many ways to compute the p-value. This approach below uses the observed and expected (Predicted) values directly with the `chisquare` function.

In [5]:
from scipy.stats import chisquare

chisquare(df['Observed'], df['Predicted'], ddof=dof)

Power_divergenceResult(statistic=419.4679873479184, pvalue=1.342780486250113e-90)

But, since we already have the $\chi^2$ value and degrees of freedom, we can use the `chi2.cdf` function.

In [6]:
from scipy.stats import chi2

1 - chi2.cdf(chi_sq, dof)

0.0

## Question 1.2

We are testing if two variables, `I` and `J` are independent givent a third, `K`.

### Question 1.2.1 INDEP(MD, RN | Complaint)?

- I = MD
- J = RN
- K = Complaint

In [7]:
def get_expected(r, i, j, k):
    Y_ij_ = df[(df[i]==r[i]) & (df[j]==r[j])]['Observed'].sum()
    Y___k = df[df[k]==r[k]]['Observed'].sum()
    n = df['Observed'].sum()
    
    expected = Y_ij_ * Y___k / n
    return expected

df['Predicted'] = df.apply(lambda r: get_expected(r, 'MD', 'RN', 'Complaint'), axis=1)
df

,MD,RN,Complaint,Observed,Predicted
0,George,Jim,Yes,53,47.282799
1,George,Jim,No,424,429.717201
2,George,Jill,Yes,11,4.758017
3,George,Jill,No,37,43.241983
4,Smith,Jim,Yes,0,1.784257
5,Smith,Jim,No,18,16.215743
6,Smith,Jill,Yes,4,14.174927
7,Smith,Jill,No,139,128.825073


In [8]:
def get_dof(i, j, k):
    I = len(df[i].unique())
    J = len(df[j].unique())
    K = len(df[k].unique())
    IJK = I * J * K
    
    dof = (IJK - 1) - ((I-1) + (J-1) + (K-1))
    
    return dof

chisquare(df['Observed'], df['Predicted'], ddof=get_dof('MD', 'RN', 'Complaint'))

Power_divergenceResult(statistic=19.945072739172545, pvalue=0.0001742500389547796)

### Question 1.2.2 INDEP(MD, COMPLAINT | RN)?

- I = MD
- J = Complaint
- K = RN

In [9]:
df['Predicted'] = df.apply(lambda r: get_expected(r, 'MD', 'Complaint', 'RN'), axis=1)
df

,MD,RN,Complaint,Observed,Predicted
0,George,Jim,Yes,53,46.180758
1,George,Jim,No,424,332.645773
2,George,Jill,Yes,11,17.819242
3,George,Jill,No,37,128.354227
4,Smith,Jim,Yes,0,2.886297
5,Smith,Jim,No,18,113.287172
6,Smith,Jill,Yes,4,1.113703
7,Smith,Jill,No,139,43.712828


In [10]:
chisquare(df['Observed'], df['Predicted'], ddof=get_dof('MD', 'Complaint', 'RN'))

Power_divergenceResult(statistic=391.950049303092, pvalue=1.2268386569229332e-84)

### Question 1.2.3 INDEP(RN, COMPLAINT | MD)?

- I = RN
- J = Complaint
- K = MD

In [11]:
df['Predicted'] = df.apply(lambda r: get_expected(r, 'RN', 'Complaint', 'MD'), axis=1)
df

,MD,RN,Complaint,Observed,Predicted
0,George,Jim,Yes,53,40.561224
1,George,Jim,No,424,338.265306
2,George,Jill,Yes,11,11.479592
3,George,Jill,No,37,134.693878
4,Smith,Jim,Yes,0,12.438776
5,Smith,Jim,No,18,103.734694
6,Smith,Jill,Yes,4,3.520408
7,Smith,Jill,No,139,41.306122


In [12]:
chisquare(df['Observed'], df['Predicted'], ddof=get_dof('RN', 'Complaint', 'MD'))

Power_divergenceResult(statistic=410.84182247817546, pvalue=9.923443883578908e-89)

## Question 1.3

We can also use `log-linear` modeling to test for independence. This method uses counts from the contingency table as the dependent variable, `y`, and indicators (1 or 0) for the variables as the independent variables, `X`.

In the model summary, pay attention to the `deviance` and `log-likelihood`; these are measures of `goodness-of-fit` (how well the model fits to the data). For deviance, a lower value is better and for log-likehood, a higher value is better.

In [13]:
df = pd.DataFrame({
    'MD': [0] * 4 + [1] * 4,
    'RN': [0, 0, 1, 1] * 2,
    'Complaint': [1, 0] * 4,
    'Observed': [53, 424, 11, 37, 0, 18, 4, 139]
})

df

,MD,RN,Complaint,Observed
0,0,0,1,53
1,0,0,0,424
2,0,1,1,11
3,0,1,0,37
4,1,0,1,0
5,1,0,0,18
6,1,1,1,4
7,1,1,0,139


### Question 1.3.1 Observed ~ (RN + MD + Complaint)^2

This model is called the `homogeneous model` since all pairwise interactions/assocations are included. In modeling, we start with the homogeneous model and interactively and step-wise remove interaction/assocation terms while observing the change in `goodness-of-fit`. If removing an interaction term improves the goodness-of-fit or does not impact it, then we permanently remove that term (why? we are biased towards model `parsimony`).

In [14]:
from patsy import dmatrices

formula = 'Observed ~ (RN + MD + Complaint)**2'
y, X = dmatrices(formula, df, return_type='dataframe')
y = y.values.reshape(1, -1)[0]

X

,Intercept,RN,MD,Complaint,RN:MD,RN:Complaint,MD:Complaint
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,1.0,0.0,1.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,1.0,1.0,0.0,0.0,1.0
5,1.0,0.0,1.0,0.0,0.0,0.0,0.0
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,0.0,1.0,0.0,0.0


In [15]:
y

array([ 53., 424.,  11.,  37.,   0.,  18.,   4., 139.])

In [16]:
import statsmodels.api as sm

poisson = sm.GLM(y, X, family=sm.families.Poisson())
result = poisson.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                    8
Model:                            GLM   Df Residuals:                        1
Model Family:                 Poisson   Df Model:                            6
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -19.298
Date:                Tue, 08 Mar 2022   Deviance:                      0.41586
Time:                        11:42:36   Pearson chi2:                    0.217
No. Iterations:                     8   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        6.0502      0.049    124.637      0.000       5.955       6.145
RN              -2.4447      0.171    -14.264      0.000      -2.781      -2.109
MD              -3.1709      0.241    -13.177      0.000      -3.643      -2.699
Complaint       -2.0837      0.146    -14.306      0.000      -2.369      -1.798
RN:MD            4.5013      0.302     14.896      0.000       3.909       5.094
RN:Complaint     0.8939      0.366      2.439      0.015       0.176       1.612
MD:Complaint    -2.4109      0.600     -4.021      0.000      -3.586      -1.236
================================================================================
"""

### Question 1.3.2 Observed ~ RN + MD + Complaint + RN:Complaint + MD:Complaint

We have removed the `RN:MD` interaction; this model is a worse fit (Deviance=382) than the homogenous model (Deviance=0.42).

In [17]:
formula = 'Observed ~ RN + MD + Complaint + RN:Complaint + MD:Complaint'
y, X = dmatrices(formula, df, return_type='dataframe')
y = y.values.reshape(1, -1)[0]

In [18]:
poisson = sm.GLM(y, X, family=sm.families.Poisson())
result = poisson.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                    8
Model:                            GLM   Df Residuals:                        2
Model Family:                 Poisson   Df Model:                            5
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -210.09
Date:                Tue, 08 Mar 2022   Deviance:                       382.00
Time:                        11:42:37   Pearson chi2:                     388.
No. Iterations:                     7   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        5.7982      0.053    109.312      0.000       5.694       5.902
RN              -0.9208      0.089    -10.331      0.000      -1.096      -0.746
MD              -1.0772      0.092    -11.657      0.000      -1.258      -0.896
Complaint       -1.8886      0.150    -12.562      0.000      -2.183      -1.594
RN:Complaint    -0.3414      0.306     -1.117      0.264      -0.941       0.258
MD:Complaint    -1.6954      0.524     -3.238      0.001      -2.722      -0.669
================================================================================
"""

### Question 1.3.3 Observed ~ RN + MD + Complaint + RN:MD + RN:Complaint

We have removed the `Complaint:MD` term; again, this model is a worse fit than the homogenous one.

In [19]:
formula = 'Observed ~ RN + MD + Complaint + RN:MD + RN:Complaint'
y, X = dmatrices(formula, df, return_type='dataframe')
y = y.values.reshape(1, -1)[0]

In [20]:
poisson = sm.GLM(y, X, family=sm.families.Poisson())
result = poisson.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                    8
Model:                            GLM   Df Residuals:                        2
Model Family:                 Poisson   Df Model:                            5
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -29.641
Date:                Tue, 08 Mar 2022   Deviance:                       21.101
Time:                        11:42:37   Pearson chi2:                     22.3
No. Iterations:                     6   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        6.0543      0.048    125.192      0.000       5.959       6.149
RN              -2.2649      0.154    -14.737      0.000      -2.566      -1.964
MD              -3.2771      0.240    -13.649      0.000      -3.748      -2.807
Complaint       -2.1210      0.145    -14.591      0.000      -2.406      -1.836
RN:MD            4.3688      0.292     14.943      0.000       3.796       4.942
RN:Complaint    -0.3414      0.306     -1.117      0.264      -0.941       0.258
================================================================================
"""

### Question 1.3.4 Observed ~ RN + MD + Complaint + RN:MD + MD:Complaint

We have removed the `Complaint:RN` interaction term. Notice how the deviance is the smallest one (compared to removing other interaction terms) at 5.74. Although this model has a higher deviance than the homogenous one, the increase is very small. Is it worth it to remove this interaction term to gain 1 degree of freedom (a simpler model)? 

In [21]:
formula = 'Observed ~ RN + MD + Complaint + RN:MD + MD:Complaint'
y, X = dmatrices(formula, df, return_type='dataframe')
y = y.values.reshape(1, -1)[0]

In [22]:
poisson = sm.GLM(y, X, family=sm.families.Poisson())
result = poisson.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                    8
Model:                            GLM   Df Residuals:                        2
Model Family:                 Poisson   Df Model:                            5
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -21.960
Date:                Tue, 08 Mar 2022   Deviance:                       5.7399
Time:                        11:42:37   Pearson chi2:                     6.19
No. Iterations:                     8   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        6.0375      0.049    124.257      0.000       5.942       6.133
RN              -2.2963      0.151    -15.165      0.000      -2.593      -2.000
MD              -3.1723      0.241    -13.164      0.000      -3.645      -2.700
Complaint       -1.9745      0.133    -14.802      0.000      -2.236      -1.713
RN:MD            4.3688      0.292     14.943      0.000       3.796       4.942
MD:Complaint    -1.6954      0.524     -3.238      0.001      -2.722      -0.669
================================================================================
"""